Import related libraries

In [4]:
import os.path

import openai
# sokoban_solver is a custom package that contains the A* agent that is able to solve Sokoban environment levels
from src.utilities import *
import random

You must add your own OpenAI API key to the `openai-api-key.txt` file.

In [5]:
api_key = get_file_contents("openai-api-key.txt")

Initialize the OpenAI API Client with the API key

In [6]:
openai.api_key = api_key

Set your training mode for the fine-tuning process, all files related to this mode will have the name of the mode in their names.

In [14]:
train_mode = "100-input-3-epoch-0.5-temp-50-output"

Get hyperparameters for the fine-tuning process

In [8]:
hypers = read_hypers()

Convert training data to JSONL format

In [15]:
training_data_folder = f"data/training_data/{hypers[train_mode]["training_set"]}"
training_data_path = f"{training_data_folder}/training_data.jsonl"
create_jsonl_from_folder(training_data_folder, training_data_path)

['{"messages": [{"role": "system", "content": "You are a Sokoban level generator. The level is represented as a grid of characters, with specific symbols representing different elements of the game. Here are the symbols used: - `#` represents a wall. - `@` represents the player. - `$` represents a storage location. - `.` represents a crate. - ` ` (space) represents an empty space. Each level should be a solvable puzzle where all crates can be moved to storage locations. The player should start at a reasonable position within the level. There should be only one player in a level. The number of crates and storage locations should be equal."}, {"role": "user", "content": "Generate a Sokoban level with a size of 10x10."}, {"role": "assistant", "content": "#####\\n##  #\\n## $##\\n#  @ #\\n# .  #####\\n###   $  #\\n  #  .   #\\n  ### #  #\\n    #   ##\\n    ######"}]}',
 '{"messages": [{"role": "system", "content": "You are a Sokoban level generator. The level is represented as a grid of ch

Upload the training file to the API for fine-tuning purposes

In [6]:
with open(training_data_path, "r") as file:
    response = openai.File.create(file=file, purpose="fine-tune")

file_id = response["id"]
write_file_contents(f"data/client_file_ids/{train_mode}.txt", file_id)

Set the model to fine-tune. gpt-3.5-turbo-0125 is the currently recommended model.

In [7]:
model = "gpt-3.5-turbo-0125"

Create a fine-tuning job for the model with the training file

In [8]:
training_file_id = get_file_contents(f"data/client_file_ids/{train_mode}.txt")

response = openai.FineTuningJob.create(
    model=model,
    training_file=training_file_id,
    suffix=train_mode.replace("-input", "").replace("-epoch", "").replace("-temp", "").replace("-output", "")
)

fine_tuning_job_id = response["id"]
write_file_contents(f"data/fine_tuning_job_ids/{train_mode}.txt", fine_tuning_job_id)

Generate levels using the fine-tuned model

In [8]:
fine_tuned_model = get_file_contents(f"data/fine_tuning_job_ids/{train_mode}.txt")

for i in range(hypers[train_mode]["n_generations"]):
    m = random.randint(7, 35)
    n = random.randint(8, 35)
    response = openai.ChatCompletion.create(
        model=fine_tuned_model,
        messages=get_messages_for_chat_completion(m, n),
        temperature=hypers[train_mode]["temperature"],
    )
    generated_level = response.choices[0].message["content"]
    
    if not os.path.exists(f"data/generated_levels/{train_mode}"):
        os.makedirs(f"data/generated_levels/{train_mode}")
    write_file_contents(f"data/generated_levels/{train_mode}/level_{i}_{m}x{n}.txt", generated_level)

APIError: Failed to create completion as the model generated invalid Unicode output. Unfortunately, this can happen in rare situations. Consider reviewing your prompt or reducing the temperature of your request. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID req_4ff7cbea716d1c302a72013a9ade91c8 in your message.) {
  "error": {
    "message": "Failed to create completion as the model generated invalid Unicode output. Unfortunately, this can happen in rare situations. Consider reviewing your prompt or reducing the temperature of your request. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID req_4ff7cbea716d1c302a72013a9ade91c8 in your message.)",
    "type": null,
    "param": null,
    "code": null
  }
} 500 {'error': {'message': 'Failed to create completion as the model generated invalid Unicode output. Unf